Training notebook for the deeptail task without data augmentation

In [6]:
import tensorflow
import keras
keras.__version__

'2.1.2'

## Getting started 
- Download data from: https://www.kaggle.com/c/whale-categorization-playground
- Rename train.csv to targets.csv
- Rename the train directory to kaggle_train


In [7]:
import os

home_dir = os.getcwd()
fname = os.path.join(home_dir, 'targets.csv') # targets for both train and validation

f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]
lines = lines[:-1]

print(header)
print(len(lines))

['Image', 'Id']
9850


# Encoding the whale ids


In [8]:
import numpy as np
whale_ids = [line.split(',')[1] for line in lines]
whale_ids = set(whale_ids) # convert to set to remove duplicats
whale_ids = list(whale_ids) # convert back to list to make it ordered
whale_ids.remove('new_whale') # remove the new_whale since we will not train with this

print(len(whale_ids))

4250


# Preprocessing images

In [9]:
import errno 

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

Split the training images into a train and validation set, and then subdivide them into directories for each whale_id

In [10]:
import shutil
original_dataset_dir = os.path.join(home_dir, 'kaggle_train')

train_dir = os.path.join(home_dir, 'train')
#shutil.rmtree(train_dir)
mkdir_p(train_dir)

validation_dir = os.path.join(home_dir, 'validation')
#shutil.rmtree(validation_dir)
mkdir_p(validation_dir)

# Let's then create a subdirectory for each whale_id in both the train and validation directories 
# so we can use the ImageDataGenerator magic function

classes_count = len(whale_ids) # During the development phase of testing models, we dont look at all 4251 classes. 
                     # When we're ready to look at all classes we can set classes_count = len(whale_ids)

for i, whale_id in enumerate(whale_ids):
    if i < classes_count:
        mkdir_p(os.path.join(train_dir, whale_id))
        mkdir_p(os.path.join(validation_dir, whale_id))

train_image_count = 0

# Copy first 9500 files into the appropriate whale directory in train dir (only if their class is included)   
for i in range(9500):
    pic = lines[i].split(',')[0]
    whale_id = lines[i].split(',')[1]
    src = os.path.join(original_dataset_dir, pic)
    whale_id_dir = os.path.join(train_dir, whale_id)
    if os.path.isdir(whale_id_dir):
        dst = os.path.join(whale_id_dir, pic)
        shutil.copyfile(src, dst)
        train_image_count += 1

validation_image_count = 0
# copy the rest into the appropriate whale directory in validation dir    (only if their class is included)  
for i in range(9500,len(lines)):
    pic = lines[i].split(',')[0]
    whale_id = lines[i].split(',')[1]
    src = os.path.join(original_dataset_dir, pic)
    whale_id_dir = os.path.join(validation_dir, whale_id)
    if os.path.isdir(whale_id_dir):
        dst = os.path.join(whale_id_dir, pic)
        shutil.copyfile(src, dst)
        validation_image_count += 1

print(train_image_count)
print(validation_image_count)

8709
331


### Using a pre-trained convolutional base


In [11]:
from keras.applications import VGG16
image_size = (224,224) #adjustable parameter for processed image_size. Run time should 

def return_base(base_name):
    if base_name == 'VGG16':
        from keras.applications import VGG16
        return VGG16(weights='imagenet',include_top=False,
                  input_shape=(image_size[0], image_size[1], 3))
    if base_name == 'Xception':
        from keras.applications import Xception
        return Xception(weights='imagenet',include_top=False,
                  input_shape=(image_size[0], image_size[1], 3))
    else:
        print("invalid base name")
        
conv_base_full = return_base('VGG16')


In [12]:
conv_base_full.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [13]:
# here we define a reduced convolutional base, obtained by only keeping the first couple blocks
# of the original base

from keras import models
from keras import layers

conv_base_full.trainable = False
exclude_layer = False
conv_base = models.Sequential()

for layer in conv_base_full.layers:
    if layer.name == "block3_conv1":
        exclude_layer = True
    if not exclude_layer:
        layer.trainable = False
        conv_base.add(layer)
        
ll_size = conv_base.layers[-1].output_shape[2]
ll_features = conv_base.layers[-1].output_shape[-1]

In [14]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
Total para

The final feature map has shape `(ll_size, ll_size, 2048)`. That's the feature on top of which we will stick a densely-connected classifier.

We will start by simply running instances of the previously-introduced `ImageDataGenerator` to extract images as Numpy arrays as well as 
their labels. We will extract features from these images simply by calling the `predict` method of the `conv_base` model.

In [15]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

home_dir = os.getcwd()
train_dir = os.path.join(home_dir, 'train')
validation_dir = os.path.join(home_dir, 'validation')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 10

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, ll_size, ll_size, ll_features))
    labels = np.zeros(shape=(sample_count, classes_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=image_size,
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='categorical')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if (i+1) * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels


In [16]:
train_features, train_labels = extract_features(train_dir, train_image_count)
validation_features, validation_labels = extract_features(validation_dir, validation_image_count)

Found 8806 images belonging to 4250 classes.
Found 331 images belonging to 4250 classes.


In [ ]:
train_features_reshaped = np.reshape(train_features,(train_image_count, ll_size*ll_size*ll_features))
validation_features_reshaped = np.reshape(validation_features, (validation_image_count, ll_size*ll_size*ll_features))

In [18]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Conv2D(128,(3,3),input_shape=(ll_size,ll_size,ll_features)))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(256,(3,3)))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(256,(3,3)))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu')) # we can play around with 
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes_count, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 128)       147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 54, 54, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 54, 54, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 256)       295168    
_________________________________________________________________
batch_normalization_2 (Batch (None, 25, 25, 256)       1024      
_________________________________________________________________
activation_2 (Activation)    (None, 25, 25, 256)       0         
__________

In [19]:
model.compile(optimizer=optimizers.Adam(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:

history = model.fit(train_features, train_labels,
                    epochs=60,
                    batch_size=64,
                    validation_data=(validation_features, validation_labels))

Train on 8709 samples, validate on 331 samples
Epoch 1/60
 832/8709 [=>............................] - ETA: 11:19 - loss: 8.4120 - acc: 0.0000e+00

In [ ]:
model.save('weights/no_aug_0.h5')

In [ ]:
from keras.models import load_model

model = load_model('weights/name_that_whale_702.h5')
model.summary()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()